In [1]:
import sys

import datetime
import pickle

import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity



import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:

df_states_codes =pd.read_csv('../Data/Mine/US_states_codes.csv')

df_states_codes

In [ ]:

df_crime_tot =pd.read_csv('../Data/Mine/Crime_TOTAL_US_1960_2014_Added_per100000.csv')

df_crime_tot



In [ ]:

df_crime_state_by_state =pd.read_csv('../Data/Mine/CrimeStatebyState_1960-2014_added_per100000.csv')

df_crime_state_by_state

In [ ]:
df_merged = pd.merge(df_crime_state_by_state, df_states_codes, on='State', how='left')

df_merged

In [ ]:
df_merged.to_csv(path_or_buf='../Data/Mine/CrimeStatebyState_1960-2014_added_per100000_abr.csv', sep=',', na_rep='NA')
print ("written: ../Data/Mine/CrimeStatebyState_1960-2014_added_per100000_abr.csv")

In [ ]:
df_sected_crime = df_crime_state_by_state[df_crime_state_by_state['State']== 'Alabama' ]

df_sected_crime

In [ ]:
for state in df_crime_state_by_state.State.unique():
    print (state)

In [ ]:
# Create and style traces

list_interesting_states = ['Alabama', 'California', 'Illinois', 'Massachusetts', 'Missouri', 'New York', 'Tennessee', 'Wyoming']

data = []

trace0 = go.Scatter(
    x = df_crime_tot['Year'],
    y = df_crime_tot['Murder_per100000'],
    name = 'tot. US crime',
    line = dict(
        color = ('rgb(0, 0, 0)'),
        width = 4)
)
data.append(trace0)

## i remove DC for now, since its murder rate is one order of magnitude higher than anywhere else
df_sected_crime_no_DC = df_crime_state_by_state[df_crime_state_by_state['State']!= 'District of Columbia' ]


#for state in df_crime_state_by_state.State.unique():
for state in list_interesting_states:   
    
    
    
    df_sected_crime = df_sected_crime_no_DC[df_sected_crime_no_DC['State']== state ]
    
    
    trace1 = go.Scatter(
        x = df_sected_crime['Year'],
        y = df_sected_crime['Murder_per100000'],
        name = state,

    )
    data.append(trace1)

    
# Edit the layout
layout = dict(
#     title = 'Average High and Low Temperatures in New York',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Average number of murders per 100,000 people'),
              )

fig = dict(data=data, layout=layout)


plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="time_evol_tot_crime" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_tot_crime.html', validate=True)




In [ ]:
offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_"+str(year),
             image_width=2000, image_height=1000, 
             filename='/your_path/'+"map_us_crime_"+str(year)+'.html', validate=True)


In [ ]:



year = 1982

df_sected_crime = df_merged[df_merged['State']!= 'District of Columbia' ]

df_sected_crime = df_merged[df_merged['Year']== year ]
  

for col in df_sected_crime.columns:
    df_sected_crime[col] = df_sected_crime[col].astype(str)
    

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


df_sected_crime['text'] = df_sected_crime['State'] + '<br>' +\
    'Pop: ' + df_sected_crime['Population'] + '<br>' +\
    'Tot. Violent crimes: '+df_sected_crime['Violent_crime_total'] + '<br>' +\
    'Murder rate: '+df_sected_crime['Murder_per100000']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_sected_crime['State_code'],
        z = df_sected_crime['Murder_per100000'].astype(float),
        locationmode = 'USA-states',
        text = df_sected_crime['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',  # while lines to separate states
                width = 2
            ) ),
        colorbar = dict(
            title = "Murder rate per 100,000 people",
           # tickvals= [1,5,10,20],
            #ticktext= [1,5,10,20],
           # colorscale = scl,
#             cmin=1,
#             cmax=20,
            #color = df_upgrading.month
            )
        ) ]




layout = dict(
        title = year,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
#              showlakes = True,
#             lakecolor = 'rgb(73, 216, 230)'  
            ),
             )
    
fig = dict( data=data, layout=layout )

plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_"+str(year) ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/'+"map_us_crime_"+str(year)+'.html', validate=True)



In [ ]:
## US crime map + slider:

### colorscale    
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]



    
data_slider = []


#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in df_merged.Year.unique():
    print (year)
 
   # df_sected_crime = df_merged[df_merged['State']!= 'District of Columbia' ]
    df_sected_crime = df_merged[df_merged['Year']== year ]
    
    
    for col in df_sected_crime.columns:
        df_sected_crime[col] = df_sected_crime[col].astype(str)

    df_sected_crime['text'] = df_sected_crime['State'] + '<br>' +\
    'Pop: ' + df_sected_crime['Population'] + '<br>' +\
    'Tot. Violent crimes: '+df_sected_crime['Violent_crime_total'] + '<br>' +\
    'Murder rate: '+df_sected_crime['Murder_per100000']

  
    data_one_year = dict(
                        type='choropleth',
                        locations = df_sected_crime['State_code'],
                        z=df_sected_crime['Murder_per100000'].astype(float),
                        locationmode='USA-states',
                        colorscale = scl,
                        text = df_sected_crime['text'],
                        colorbar = dict(
                            title = "Murder rate per 100,000 people")
                        )

    data_slider.append(data_one_year)  # I add the dictionary to the list of dictionaries for the slider
  
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1960))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_slider.html', validate=True)


In [ ]:
## NOTE:  converter of code into html:  
##  http://hilite.me/

## if the background of the blog is use either 'native' or 'tango' as styles in the drop-down menu

In [ ]:

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')


# df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
# limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
# colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
# cities = []
# scale = 5000

# for i in range(len(limits)):
#     lim = limits[i]
#     df_sub = df[lim[0]:lim[1]]
#     city = dict(
#         type = 'scattergeo',
#         locationmode = 'USA-states',
#         lon = df_sub['lon'],
#         lat = df_sub['lat'],
#         text = df_sub['text'],
#         marker = dict(
#             size = df_sub['pop']/scale,
#             # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
#             color = colors[i],
#             line = dict(width=0.5, color='rgb(40,40,40)'),
#             sizemode = 'area'
#         ),
#         name = '{0} - {1}'.format(lim[0],lim[1]) )
#     cities.append(city)

# layout = dict(
#         title = '2014 US city populations<br>(Click legend to toggle traces)',
#         showlegend = True,
#         geo = dict(
#             scope='usa',
#             projection=dict( type='albers usa' ),
#             showland = True,
#             landcolor = 'rgb(217, 217, 217)',
#             subunitwidth=1,
#             countrywidth=1,
#             subunitcolor="rgb(255, 255, 255)",
#             countrycolor="rgb(255, 255, 255)"
#         ),
#     )

# fig = dict(data=cities, layout=layout)


# plotly.offline.iplot(fig)

In [ ]:
#df.head()

In [ ]:
#df_crime_cities.head()

In [ ]:
#sorted(df_crime_cities['1985'].unique())

In [12]:
###  crime map - CITIES  (fixed color)

df_crime_cities = pd.read_csv('../Data/Mine/LocalCrimeTrendsInOneVar_crime_rate_vs_time_cities_500000plus_1985_2014.csv')


# i need the all-time maximum value to rescale the city circles accordingly
max_value_years = 0
i = 1985
while i <=2014:   
    value = df_crime_cities[str(i)].max()
    if  value > max_value_years:
        max_value_years = value
    i  +=1





cities = []  # data structure for the plot (list of dict)


year = 1985

df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
df_for_plotting.dropna(subset=[str(year)], inplace=True)

df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)


city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_for_plotting['lon'],
    lat = df_for_plotting['lat'],
    text = df_for_plotting['text'],
    marker = dict(
        size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
        # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
        color = "#ff3300",
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
   
)
cities.append(city)

layout = dict(
        title = 'Crime rate in major US cities in '+str(year),
       # showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=cities, layout=layout)


plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html', validate=True)


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html'

In [ ]:
df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False)  # add a new column to rank rows by the value of a given column
df_for_plotting.sort_values(by=[str(year)])

In [ ]:
#ffffff   white-red
#ffe6e6
#ffcccc
#ffb3b3
#ff9999
#ff8080
#ff6666
#ff4d4d
#ff3333
#ff1a1a
#ff0000
#e60000
#cc0000
#b30000
#990000
#800000
#660000

#4d0000     dark-red


In [4]:
###  crime map - CITIES  
### adding now bins for different sizes of circles-crime (calculated with the all-time lows and highs)




year = 2005


### select year:
df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
df_for_plotting.dropna(subset=[str(year)], inplace=True)
df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)

# add a new column to rank rows by the value of a given column
df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False)  





#### i need the all-time maximum value to rescale/normalize 
#    the areas of the circles consistently across years
lista_all_values = []
max_value_years = 0
i = 1985
while i <=2014:   
    value = df_crime_cities[str(i)].max()
    lista_all_values += df_crime_cities[str(i)].dropna().tolist()  # remove the NANs first or the np.percentile wont work
    if  value > max_value_years:
        max_value_years = value
    i  +=1
list_values_year = df_crime_cities[str(year)].dropna().tolist()  



 
list_pairs_ranks = [[1,9],[9,18],[18,27],[27,36]]     # add an extra rank at the end

colors = ["#4d0000","#ff1a1a","#ff9999","#ffe6e6"]

all_cities = []  # data object for plotting a given year
cont_bines = 0
for pair_values in list_pairs_ranks:   
    #print ("-------",pair_values)
    
    minimo = pair_values[0]
    maximo = pair_values[1]   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
    
   




    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = colors[cont_bines],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = 'ranked {0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    all_cities.append(city_group)
    cont_bines +=1
    
    
layout = dict(
        title = 'Crime rate in major US cities in '+str(year),
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=all_cities, layout=layout)


plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html', validate=True)


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html'

In [11]:
## US CITIES crime map + slider (fixed color):



list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]






    
data_slider = []
#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)

    
    
    data_one_year = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_for_plotting['lon'],
        lat = df_for_plotting['lat'],
        text = df_for_plotting['text'],
        marker = dict(
            size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = "#ff3300",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),

    )
    data_slider.append(data_one_year)
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1985))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html'

In [10]:

## US CITIES crime map + slider (fixed color):



list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]






    
data_slider = []
#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)


    data_one_year = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_for_plotting['lon'],
        lat = df_for_plotting['lat'],
        text = df_for_plotting['text'],
        marker = dict(
            size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = "#ff3300",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),

    )
    
    
    
    
    
    
    data_slider.append(data_one_year)
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1985))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html'

In [13]:
### WORK IN PROGRESS




list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]



lista_traces = []  # tot 

trace1_list = []   # one color, all years

trace2_list = []  # another color, all years
trace3_list = []  # another color, all years
trace4_list = []  # another color, all years



#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)
    df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False) ## add column with city ranking for that year
    
    
    ### for top ranked cities:
    minimo = 1
    maximo = 9   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#4d0f00",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = 'TOP ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace1_list.append(city_group)
    
    
    
    
    
    
    ### for sencond-ranked group of cities:
    minimo = 9
    maximo = 18   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#b32400",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '2nd ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace2_list.append(city_group)
    
    
      
        
    ### for sencond-ranked group of cities:
    minimo = 18
    maximo = 27   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#ff704d",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '3rd ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace3_list.append(city_group)
    
    
    
        
    ### for sencond-ranked group of cities:
    minimo = 27
    maximo = 35   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#ffd6cc",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '4th ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace4_list.append(city_group)
    


lista_traces += trace1_list
lista_traces += trace2_list
lista_traces += trace3_list
lista_traces += trace4_list
num_steps = 30   # as many as years




steps = []
for i in range(num_steps):
    # Start by hiding all traces
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(lista_traces)],
        label='Year {}'.format(i + 1985)
    )
    
    # Enable the N traces we want to see
    step['args'][1][i] = True
    step['args'][1][i+num_steps] = True
    step['args'][1][i+num_steps*2] = True
    step['args'][1][i+num_steps*3] = True
   
    
    # Add step to step list
    steps.append(step)

    
      



sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=lista_traces, layout=layout)


# ----------------



 

# # let's create the steps for the slider
# steps = []
# for i in range(len(data_slider)):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(data_slider)],
#                 label='Year {}'.format(i + 1985))
#     step['args'][1][i] = True
#     steps.append(step)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html'

In [ ]:
len(list_years)

In [ ]:
# # example slider with multiple traces

# lista_traces = []
# num_steps = 5  # years

# trace1_list = [  # as many (red) traces as steps (years)
#     go.Scatter(y=[1, 2, 3], x=[1, 2, 3], visible=True, line={'color': 'red'}), 
#     go.Scatter(y=[10, 11, 12], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[20, 21, 22], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[30, 31, 32], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[40, 41, 42], x=[1, 2, 3], visible=True, line={'color': 'red'}) 
# ]
# lista_traces += trace1_list


# trace2_list = [
#     go.Scatter(y=[30, 29, 28], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[10,9, 8], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[40, 39, 38], x=[1, 2, 3], visible=True, line={'color': 'blue'}),
#     go.Scatter(y=[20, 19, 18], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[9, 8, 7], x=[1, 2, 3], visible=True, line={'color': 'blue'})  
# ]
# lista_traces += trace2_list




# trace3_list = [
#     go.Scatter(y=[10, 31, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 25, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 20, 21], x=[1, 2, 3], visible=True, line={'color': 'green'}),
#     go.Scatter(y=[10, 15, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 10, 20], x=[1, 2, 3], visible=True, line={'color': 'green'})  
# ]
# lista_traces += trace3_list




# trace4_list = [
#     go.Scatter(y=[15, 31, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[15, 25, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[15, 20, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),
#     go.Scatter(y=[15, 15, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[50, 10, 20], x=[1, 2, 3], visible=True, line={'color': 'grey'})  
# ]
# lista_traces += trace4_list




# fig = go.Figure(data=lista_traces)

# steps = []
# for i in range(num_steps):
#     # Hide all traces
#     step = dict(
#         method = 'restyle',  
#         args = ['visible', [False] * len(fig.data)],
#     )
#     # Enable the N traces we want to see
#     step['args'][1][i] = True
#     step['args'][1][i+num_steps] = True
#     step['args'][1][i+num_steps*2] = True
#     step['args'][1][i+num_steps*3] = True
   
    
#     # Add step to step list
#     steps.append(step)

# sliders = [dict(
#     steps = steps,
# )]

# fig.layout.sliders = sliders

# iplot(fig, show_link=False)

In [ ]:
# ## working example map + slider  

# plotly.offline.init_notebook_mode()
# df_2011 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')


# ## original dataset is just for one year
# data_slider = [dict(type='choropleth',
#              locations = df_2011['code'].astype(str),
#              z=df_2011['total exports'].astype(float),
#              locationmode='USA-states')]


# # let's create some additional, random data, to fill up different years from the original one-year dataset
# for i in range(5):
#     data_slider.append(data_slider[0].copy())
#     data_slider[-1]['z'] = data_slider[0]['z'] * np.random.rand(*data_slider[0]['z'].shape)

    
    
# # let's create the steps for the slider
# steps = []
# for i in range(len(data_slider)):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(data_slider)],
#                 label='Year {}'.format(i + 1980))
#     step['args'][1][i] = True
#     steps.append(step)

# sliders = [dict(active=0,
#                 pad={"t": 1},
#                 steps=steps)]    
# layout = dict(geo=dict(scope='usa',
#                        projection={'type': 'albers usa'}),
#               sliders=sliders)

# fig = dict(data=data_slider, 
#            layout=layout)
# plotly.offline.iplot(fig)



# offline.plot(fig, auto_open=True, image = 'png', image_filename="map_slider" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_slider.html', validate=True)


In [ ]:
# ### working example sine function with slider:

# ### i build the data list (one per different position that the slider will have)
# data = [dict(
#         visible = False,
#         line=dict(color='#00CED1', width=6),
#         name = '𝜈 = '+str(step),
#         x = np.arange(0,100,0.01),
#         y = np.sin(step*np.arange(0,100,0.01))) for step in np.arange(0,5,0.1)]



# data[1]['visible'] = True



# ### i visualize the plot with slider
# steps = []
# for i in range(len(data)):
#     step = dict(
#         method = 'restyle',  
#         args = ['visible', [False] * len(data)],
#     )
#     step['args'][1][i] = True # Toggle i'th trace to "visible"
#     steps.append(step)

# sliders = [dict(
#     active = 10,
#     currentvalue = {"prefix": "Frequency: "},
#     pad = {"t": 50},
#     steps = steps
# )]

# layout = dict(sliders=sliders)

# fig = dict(data=data, layout=layout)

# #py.iplot(fig, filename='Sine Wave Slider')



# offline.plot(fig, auto_open=True, image = 'png', image_filename="sine_slider" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/sine_slider.html', validate=True)



In [ ]:
# ### example US map by state

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

# for col in df.columns:
#     df[col] = df[col].astype(str)

# scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
#             [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

# # df['text'] = df['state'] + '<br>' +\
# #     'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
# #     'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
# #     'Wheat '+df['wheat']+' Corn '+df['corn']

# data = [ dict(
#         type='choropleth',
#         colorscale = scl,
#         autocolorscale = False,
#         locations = df['code'],
#         z = df['total exports'].astype(float),
#         locationmode = 'USA-states',
#        # text = df['text'],
#         marker = dict(
#             line = dict (
#                 color = 'rgb(255,255,255)',
#                 width = 2
#             ) ),
#         colorbar = dict(
#             title = "Millions USD")
#         ) ]

# layout = dict(
#         title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
#         geo = dict(
#             scope='usa',
#             projection=dict( type='albers usa' ),
#             showlakes = True,
#             lakecolor = 'rgb(255, 255, 255)'),
#              )
    
# fig = dict( data=data, layout=layout )




# offline.plot(fig, auto_open=True, image = 'png', image_filename="d3-cloropleth-map" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/d3-cloropleth-map.html', validate=True)

